# 이상치 분석 및 파생변수 효과 분석

**목표**:
- 1등 팀이 제거한 이상치 근거 검증
- 추가 이상치 후보 발견 및 분석
- 파생변수(CDH, CDD, THI, WCT 등)의 효과 분석
- 주기성 특징(sin/cos)의 효과 시각화

## 📦 1. 라이브러리 및 데이터 로드

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from scipy import stats
from datetime import datetime

warnings.filterwarnings('ignore')

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False
sns.set_style('whitegrid')

plt.rcParams['figure.figsize'] = (14, 6)

print("라이브러리 임포트 완료")

In [ ]:
# 데이터 로드
DATA_DIR = './data'
train = pd.read_csv(os.path.join(DATA_DIR, 'train.csv'), encoding='utf-8-sig')
building_info = pd.read_csv(os.path.join(DATA_DIR, 'building_info.csv'), encoding='utf-8-sig')

# 날짜 변환
train['일시'] = pd.to_datetime(train['일시'], format='%Y%m%d %H')

print(f"Train shape: {train.shape}")
print(f"Building info shape: {building_info.shape}")

## 🔍 2. 이상치 탐지 및 분석

### 2.1 통계적 이상치 탐지 (IQR 방법)

In [ ]:
def detect_outliers_iqr(data, column='전력소비량(kWh)', multiplier=1.5):
    """
    IQR 방법을 사용한 이상치 탐지
    """
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - multiplier * IQR
    upper_bound = Q3 + multiplier * IQR
    
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    
    return outliers, lower_bound, upper_bound

# 전체 데이터에 대한 이상치 탐지
outliers, lower, upper = detect_outliers_iqr(train)

print(f"IQR 방법 이상치 탐지 결과:")
print(f"  - 하한: {lower:.2f} kWh")
print(f"  - 상한: {upper:.2f} kWh")
print(f"  - 이상치 개수: {len(outliers):,}개 ({len(outliers)/len(train)*100:.2f}%)")
print(f"  - 이상치 건물 수: {outliers['건물번호'].nunique()}개")

### 2.2 건물별 이상치 분석

In [ ]:
# 건물별 이상치 개수
outlier_by_building = []

for building in train['건물번호'].unique():
    building_data = train[train['건물번호'] == building]
    building_outliers, _, _ = detect_outliers_iqr(building_data)
    
    outlier_by_building.append({
        '건물번호': building,
        '전체 데이터': len(building_data),
        '이상치 개수': len(building_outliers),
        '이상치 비율(%)': len(building_outliers) / len(building_data) * 100
    })

outlier_summary = pd.DataFrame(outlier_by_building).sort_values('이상치 개수', ascending=False)

print("\n=== 이상치가 많은 상위 10개 건물 ===")
print(outlier_summary.head(10))

In [ ]:
# 이상치 비율 시각화
plt.figure(figsize=(16, 6))
plt.bar(outlier_summary['건물번호'].astype(str), outlier_summary['이상치 비율(%)'], 
       alpha=0.7, edgecolor='black')
plt.axhline(y=5, color='r', linestyle='--', label='5% 기준선')
plt.title('건물별 이상치 비율', fontsize=16, fontweight='bold')
plt.xlabel('건물번호', fontsize=12)
plt.ylabel('이상치 비율 (%)', fontsize=12)
plt.xticks(rotation=90)
plt.legend()
plt.grid(alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

### 2.3 1등 팀이 제거한 이상치 검증

In [ ]:
# 1등 팀의 이상치 제거 규칙 재현
removed_outliers_analysis = []

# 규칙 1: 특정 건물의 낮은 값 제거
rules_lt = [
    (25, 0, "eq"), (70, 200, "lt"),
    (44, 800, "lt"), (90, 800, "lt"), (42, 2000, "lt"), (17, 1000, "lt"),
    (68, 600, "lt"), (72, 600, "lt"), (80, 600, "lt"), (92, 200, "lt"),
    (98, 500, "lt"), (97, 500, "lt"), (78, 400, "lt"), (26, 300, "lt"),
]

for building, threshold, op in rules_lt:
    building_data = train[train['건물번호'] == building]
    
    if op == "lt":
        removed = building_data[building_data['전력소비량(kWh)'] < threshold]
    elif op == "eq":
        removed = building_data[building_data['전력소비량(kWh)'] == threshold]
    
    if len(removed) > 0:
        removed_outliers_analysis.append({
            '건물번호': building,
            '규칙': f"{op} {threshold}",
            '제거된 데이터 수': len(removed),
            '제거 비율(%)': len(removed) / len(building_data) * 100,
            '제거된 값 범위': f"{removed['전력소비량(kWh)'].min():.2f} - {removed['전력소비량(kWh)'].max():.2f}"
        })

removed_df = pd.DataFrame(removed_outliers_analysis)
print("\n=== 1등 팀 이상치 제거 규칙 분석 ===")
print(removed_df)

In [ ]:
# 특정 건물의 이상치 시각화 (예: 건물 25)
example_buildings = [25, 70, 44, 90, 42]

fig, axes = plt.subplots(len(example_buildings), 1, figsize=(16, 4*len(example_buildings)))

for idx, building in enumerate(example_buildings):
    building_data = train[train['건물번호'] == building].sort_values('일시')
    
    # 해당 건물의 제거 규칙 찾기
    rule = [r for r in rules_lt if r[0] == building]
    if rule:
        _, threshold, op = rule[0]
        
        axes[idx].plot(building_data['일시'], building_data['전력소비량(kWh)'], 
                      linewidth=1, alpha=0.7, label='실제 데이터')
        
        if op == 'lt':
            axes[idx].axhline(y=threshold, color='r', linestyle='--', linewidth=2, 
                            label=f'임계값: {threshold} kWh (이하 제거)')
            outlier_mask = building_data['전력소비량(kWh)'] < threshold
        elif op == 'eq':
            outlier_mask = building_data['전력소비량(kWh)'] == threshold
        
        if outlier_mask.any():
            axes[idx].scatter(building_data[outlier_mask]['일시'], 
                            building_data[outlier_mask]['전력소비량(kWh)'],
                            color='red', s=50, zorder=5, alpha=0.7, label='이상치')
        
        axes[idx].set_title(f'건물 {building} - 이상치 분석', fontsize=14, fontweight='bold')
        axes[idx].set_xlabel('일시', fontsize=11)
        axes[idx].set_ylabel('전력소비량 (kWh)', fontsize=11)
        axes[idx].legend()
        axes[idx].grid(alpha=0.3)

plt.tight_layout()
plt.show()

### 2.4 Z-Score 기반 이상치 탐지

In [ ]:
# 건물별 Z-Score 계산
train['z_score'] = train.groupby('건물번호')['전력소비량(kWh)'].transform(
    lambda x: np.abs(stats.zscore(x))
)

# Z-Score > 3인 이상치
z_outliers = train[train['z_score'] > 3]

print(f"\nZ-Score 기반 이상치 (|Z| > 3):")
print(f"  - 이상치 개수: {len(z_outliers):,}개 ({len(z_outliers)/len(train)*100:.2f}%)")
print(f"  - 이상치 건물 수: {z_outliers['건물번호'].nunique()}개")

# Z-Score 분포 시각화
plt.figure(figsize=(14, 6))
plt.hist(train['z_score'], bins=100, edgecolor='black', alpha=0.7)
plt.axvline(x=3, color='r', linestyle='--', linewidth=2, label='Z-Score = 3')
plt.title('전력소비량 Z-Score 분포', fontsize=16, fontweight='bold')
plt.xlabel('|Z-Score|', fontsize=12)
plt.ylabel('빈도', fontsize=12)
plt.xlim(0, 10)
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## 🔧 3. 파생변수 효과 분석

### 3.1 파생변수 생성

In [ ]:
# 1. CDH (Cooling Degree Hour) - 냉방도시
def cooling_degree_hour(temperature, window=12, base_temp=26):
    cdhs = []
    temps = temperature.values
    for i in range(len(temps)):
        if i < window:
            cdh = np.sum(np.maximum(temps[:i+1] - base_temp, 0))
        else:
            cdh = np.sum(np.maximum(temps[i-window+1:i+1] - base_temp, 0))
        cdhs.append(cdh)
    return cdhs

# 건물별 CDH 계산 (샘플로 건물 1개만)
sample_building = train[train['건물번호'] == 1].copy()
sample_building['CDH'] = cooling_degree_hour(sample_building['기온(°C)'])

print("CDH 생성 완료 (샘플 건물 1)")
print(sample_building[['일시', '기온(°C)', 'CDH', '전력소비량(kWh)']].head(20))

In [ ]:
# 2. CDD (Cooling Degree Days)
train['excess'] = (train['기온(°C)'] - 18).clip(lower=0)
train['CDD'] = train.groupby('건물번호')['excess'].transform(
    lambda s: s.rolling(24, min_periods=1).sum()
)

# 3. THI (Temperature-Humidity Index) - 불쾌지수
train['THI'] = (9/5 * train['기온(°C)'] 
               - 0.55 * (1 - train['습도(%)'] / 100) 
               * (9/5 * train['기온(°C)'] - 26) 
               + 32)

# 4. WCT (Wind Chill Temperature) - 체감온도
v16 = train['풍속(m/s)'] ** 0.16
train['WCT'] = (13.12 
               + 0.6215 * train['기온(°C)'] 
               - 11.37 * v16 
               + 0.3965 * v16 * train['기온(°C)'])

# 5. 기온 제곱
train['기온_제곱'] = train['기온(°C)'] ** 2

# 6. 습도 제곱
train['습도_제곱'] = train['습도(%)'] ** 2

print("\n파생변수 생성 완료:")
print("  - CDD (Cooling Degree Days)")
print("  - THI (불쾌지수)")
print("  - WCT (체감온도)")
print("  - 기온_제곱")
print("  - 습도_제곱")

train = train.drop('excess', axis=1)

### 3.2 파생변수와 전력소비량 상관관계

In [ ]:
# 상관계수 계산
derived_features = ['기온(°C)', 'CDD', 'THI', 'WCT', '기온_제곱', '습도(%)', '습도_제곱']
correlation_matrix = train[derived_features + ['전력소비량(kWh)']].corr()

# 전력소비량과의 상관계수만 추출
power_corr = correlation_matrix['전력소비량(kWh)'].drop('전력소비량(kWh)').sort_values(ascending=False)

print("\n파생변수와 전력소비량 상관계수:")
print(power_corr)

# 시각화
plt.figure(figsize=(10, 6))
power_corr.plot(kind='barh', color='steelblue', edgecolor='black', alpha=0.7)
plt.title('파생변수와 전력소비량 상관계수', fontsize=16, fontweight='bold')
plt.xlabel('상관계수', fontsize=12)
plt.ylabel('변수명', fontsize=12)
plt.axvline(x=0, color='black', linewidth=1)
plt.grid(alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

In [ ]:
# 상관계수 히트맵
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt='.3f', cmap='coolwarm', center=0,
           square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('파생변수 상관계수 히트맵', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

### 3.3 파생변수 산점도

In [ ]:
# 파생변수 vs 전력소비량 산점도
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

features_to_plot = ['CDD', 'THI', 'WCT', '기온_제곱', '습도_제곱', '기온(°C)']

for idx, feature in enumerate(features_to_plot):
    # 샘플링 (너무 많은 데이터로 인한 시각화 느림 방지)
    sample_data = train.sample(n=min(10000, len(train)), random_state=42)
    
    axes[idx].scatter(sample_data[feature], sample_data['전력소비량(kWh)'], 
                     alpha=0.1, s=5)
    axes[idx].set_title(f'{feature} vs 전력소비량\n(상관: {power_corr.get(feature, 0):.4f})', 
                       fontsize=12, fontweight='bold')
    axes[idx].set_xlabel(feature, fontsize=11)
    axes[idx].set_ylabel('전력소비량 (kWh)', fontsize=11)
    axes[idx].grid(alpha=0.3)

plt.tight_layout()
plt.show()

### 3.4 주기성 특징 (Sin/Cos) 효과

In [ ]:
# 주기성 특징 생성
train['시간'] = train['일시'].dt.hour
train['day_of_year'] = train['일시'].dt.dayofyear

# 시간 주기성 (24시간)
train['sin_hour'] = np.sin(2 * np.pi * train['시간'] / 24)
train['cos_hour'] = np.cos(2 * np.pi * train['시간'] / 24)

# 연중 일자 주기성 (365일)
train['sin_doy'] = np.sin(2 * np.pi * (train['day_of_year'] - 1) / 365)
train['cos_doy'] = np.cos(2 * np.pi * (train['day_of_year'] - 1) / 365)

print("주기성 특징 생성 완료")

In [ ]:
# 주기성 특징 시각화
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# sin_hour vs 전력소비량
sample = train.sample(n=10000, random_state=42)
axes[0, 0].scatter(sample['sin_hour'], sample['전력소비량(kWh)'], alpha=0.1, s=5)
axes[0, 0].set_title('sin_hour vs 전력소비량', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('sin_hour', fontsize=12)
axes[0, 0].set_ylabel('전력소비량 (kWh)', fontsize=12)
axes[0, 0].grid(alpha=0.3)

# cos_hour vs 전력소비량
axes[0, 1].scatter(sample['cos_hour'], sample['전력소비량(kWh)'], alpha=0.1, s=5, color='orange')
axes[0, 1].set_title('cos_hour vs 전력소비량', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('cos_hour', fontsize=12)
axes[0, 1].set_ylabel('전력소비량 (kWh)', fontsize=12)
axes[0, 1].grid(alpha=0.3)

# sin_doy vs 전력소비량
axes[1, 0].scatter(sample['sin_doy'], sample['전력소비량(kWh)'], alpha=0.1, s=5, color='green')
axes[1, 0].set_title('sin_doy vs 전력소비량', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('sin_doy', fontsize=12)
axes[1, 0].set_ylabel('전력소비량 (kWh)', fontsize=12)
axes[1, 0].grid(alpha=0.3)

# cos_doy vs 전력소비량
axes[1, 1].scatter(sample['cos_doy'], sample['전력소비량(kWh)'], alpha=0.1, s=5, color='red')
axes[1, 1].set_title('cos_doy vs 전력소비량', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('cos_doy', fontsize=12)
axes[1, 1].set_ylabel('전력소비량 (kWh)', fontsize=12)
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# 시간별 평균 전력소비량과 주기성 특징 비교
hourly_avg = train.groupby('시간').agg({
    '전력소비량(kWh)': 'mean',
    'sin_hour': 'first',
    'cos_hour': 'first'
})

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 시간별 평균 전력소비량
ax1 = axes[0]
ax1.plot(hourly_avg.index, hourly_avg['전력소비량(kWh)'], marker='o', 
        linewidth=2, label='평균 전력소비량', color='steelblue')
ax1.set_title('시간별 평균 전력소비량', fontsize=14, fontweight='bold')
ax1.set_xlabel('시간', fontsize=12)
ax1.set_ylabel('평균 전력소비량 (kWh)', fontsize=12, color='steelblue')
ax1.tick_params(axis='y', labelcolor='steelblue')
ax1.grid(alpha=0.3)

# 주기성 특징 (sin/cos)
ax2 = axes[1]
ax2.plot(hourly_avg.index, hourly_avg['sin_hour'], marker='s', 
        linewidth=2, label='sin_hour', color='red')
ax2.plot(hourly_avg.index, hourly_avg['cos_hour'], marker='^', 
        linewidth=2, label='cos_hour', color='orange')
ax2.set_title('시간별 주기성 특징 (sin/cos)', fontsize=14, fontweight='bold')
ax2.set_xlabel('시간', fontsize=12)
ax2.set_ylabel('값', fontsize=12)
ax2.legend()
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 📊 4. 종합 요약

In [ ]:
print("=" * 80)
print("이상치 및 파생변수 분석 종합 요약")
print("=" * 80)

print("\n1. 이상치 탐지 결과")
print(f"   - IQR 방법: {len(outliers):,}개 ({len(outliers)/len(train)*100:.2f}%)")
print(f"   - Z-Score 방법 (|Z| > 3): {len(z_outliers):,}개 ({len(z_outliers)/len(train)*100:.2f}%)")
print(f"   - 1등 팀 제거 규칙: {len(removed_df)}개 규칙")

print("\n2. 파생변수 상관계수 (전력소비량)")
for feature, corr in power_corr.items():
    print(f"   - {feature}: {corr:.4f}")

print("\n3. 주요 발견")
print("   - 기온 제곱이 기온보다 높은 상관관계를 보임")
print("   - CDH, CDD, THI는 모두 강한 양의 상관관계")
print("   - 주기성 특징은 시간 패턴을 효과적으로 포착")

print("\n4. 다음 단계")
print("   - Feature Selection 수행")
print("   - 새로운 파생변수 개발")
print("   - 모델 학습 및 평가")

print("=" * 80)